# VAE

> Scripts to use Variational Autoencoders

In [7]:
#| default_exp vae

In [8]:
#| export
#| hide

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import MetricCollection, MeanMetric
import torch
import torch.nn as nn
from abc import ABC, abstractmethod
from torch import Tensor
from orbit_generation.architectures import Sampling, InceptionTimeVAEEncoder, InceptionTimeVAEDecoder

In [9]:
#| export
class AbstractVAE(ABC):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int):
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim

    @abstractmethod
    def encode(self, x: Tensor) -> tuple[Tensor, Tensor]:
        """Encode input to latent space."""
        pass

    @abstractmethod
    def decode(self, z: Tensor) -> Tensor:
        """Decode latent representation."""
        pass

    @abstractmethod
    def forward(self, x: Tensor) -> tuple[Tensor, Tensor, Tensor]:
        """Forward pass through the VAE."""
        pass

    @abstractmethod
    def sample(self, num_samples: int) -> Tensor:
        """Generate samples from the latent space."""
        pass

In [10]:
#| export
class BetaVAE(pl.LightningModule, AbstractVAE):
    def __init__(self, encoder: nn.Module, decoder: nn.Module, beta: float = 1.0,
                 loss_fn=None, optimizer_cls=torch.optim.Adam, lr: float = 0.001, **kwargs):
        super().__init__()
        AbstractVAE.__init__(self, encoder.seq_len, encoder.feat_dim, encoder.latent_dim)
        self.beta = beta
        self.encoder = encoder
        self.decoder = decoder
        self.sampling = Sampling()
        self.loss_fn = loss_fn if loss_fn else self.default_loss_fn
        self.optimizer_cls = optimizer_cls
        self.lr = lr
        self.save_hyperparameters(ignore=['encoder', 'decoder'])

    def setup(self, stage=None):
        self.train_metrics = MetricCollection({
            'total_loss': MeanMetric(),
            'reconstruction_loss': MeanMetric(),
            'kl_loss': MeanMetric()
        })
        self.val_metrics = MetricCollection({
            'total_loss': MeanMetric(),
            'reconstruction_loss': MeanMetric(),
            'kl_loss': MeanMetric()
        })
        self.train_metrics.to(self.device)
        self.val_metrics.to(self.device)

    def encode(self, x: Tensor) -> tuple[Tensor, Tensor]:
        return self.encoder(x)

    def decode(self, z: Tensor) -> Tensor:
        return self.decoder(z)

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor, Tensor]:
        z_mean, z_log_var = self.encode(x)
        z = self.sampling(z_mean, z_log_var)
        x_recon = self.decode(z)
        return x_recon, z_mean, z_log_var

    def reconstruction_loss_by_axis(self, x: Tensor, x_hat: Tensor, axis: int) -> Tensor:
        loss = torch.mean((x - x_hat) ** 2, dim=axis)
        return loss.sum()

    def default_loss_fn(self, x: Tensor, x_hat: Tensor, z_mean: Tensor, z_log_var: Tensor) -> tuple[Tensor, Tensor]:
        reconst_loss = sum(self.reconstruction_loss_by_axis(x, x_hat, axis) for axis in range(3))
        kl_loss = -0.5 * torch.mean(torch.sum(1 + z_log_var - z_mean ** 2 - torch.exp(z_log_var), dim=1))
        return reconst_loss, self.beta * kl_loss

    def training_step(self, batch, batch_idx):
        x = batch[0] if isinstance(batch, list) else batch
        x_hat, z_mean, z_log_var = self(x)
        reconst_loss, kl_loss = self.loss_fn(x, x_hat, z_mean, z_log_var)
        total_loss = reconst_loss + kl_loss

        # Update training metrics
        self.train_metrics['total_loss'].update(total_loss)
        self.train_metrics['reconstruction_loss'].update(reconst_loss)
        self.train_metrics['kl_loss'].update(kl_loss)

        # Log training metrics
        self.log('train_total_loss', total_loss, on_step=False, on_epoch=True)
        self.log('train_reconstruction_loss', reconst_loss, on_step=False, on_epoch=True)
        self.log('train_kl_loss', kl_loss, on_step=False, on_epoch=True)

        return total_loss

    def validation_step(self, batch, batch_idx):
        if batch is None:
            return None

        x = batch[0] if isinstance(batch, list) else batch
        x_hat, z_mean, z_log_var = self(x)
        reconst_loss, kl_loss = self.loss_fn(x, x_hat, z_mean, z_log_var)
        total_loss = reconst_loss + kl_loss

        # Update validation metrics
        self.val_metrics['total_loss'].update(total_loss)
        self.val_metrics['reconstruction_loss'].update(reconst_loss)
        self.val_metrics['kl_loss'].update(kl_loss)

        # Log validation metrics
        self.log('val_total_loss', total_loss, on_step=False, on_epoch=True)
        self.log('val_reconstruction_loss', reconst_loss, on_step=False, on_epoch=True)
        self.log('val_kl_loss', kl_loss, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        return self.optimizer_cls(self.parameters(), lr=self.lr)

    def sample(self, num_samples: int) -> Tensor:
        z = torch.randn(num_samples, self.latent_dim).to(self.device)
        return self.decode(z)

    def on_train_epoch_start(self):
        self.train_metrics.reset()

    def on_validation_epoch_start(self):
        if self.trainer.val_dataloaders:
            self.val_metrics.reset()

In [ ]:
#| export
class InceptionTimeVAE(BetaVAE):
    def __init__(self, encoder: nn.Module, decoder: nn.Module, beta: float = 1.0,
                 loss_fn=None, optimizer_cls=torch.optim.Adam, lr: float = 0.001, **kwargs):
        # Call BetaVAE's constructor
        super().__init__(encoder=encoder, decoder=decoder, beta=beta,
                         loss_fn=loss_fn, optimizer_cls=optimizer_cls, lr=lr, **kwargs)
    
    def encode(self, x: Tensor) -> tuple[Tensor, Tensor, list]:
        # Override to return z_mean, z_log_var and indices_list
        z_mean, z_log_var, indices_list = self.encoder.encode(x)
        return z_mean, z_log_var, indices_list

    def decode(self, z: Tensor, indices_list: list) -> Tensor:
        # Override to accept indices_list and pass them to the decoder
        return self.decoder.decode(z, indices_list)

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor, Tensor]:
        # Override forward pass to handle encoding and decoding with indices
        z_mean, z_log_var, indices_list = self.encode(x)
        z = self.sampling(z_mean, z_log_var)
        x_recon = self.decode(z, indices_list)
        return x_recon, z_mean, z_log_var
    
    def sample(self, num_samples: int, indices_list: list) -> Tensor:
        z = torch.randn(num_samples, self.latent_dim).to(self.device)
        return self.decode(z, indices_list)

In [12]:
#| hide
import nbdev; nbdev.nbdev_export()